<div class="usecase-section-header">Greening Laneways to reduce overland water flow events</div>

<div class="usecase-authors"><b>Authored by: </b>Alison Collins</div>

<div class="usecase-duration"><b>Duration:</b>60 mins</div>
<div class="usecase-level"><b>Level: </b>beginner</div>
<div class="usecase-skill"><b>Pre-requisite Skills: </b>Python</div>

<div class="usecase-section-header">Scenario</div>

1. As a city planner, I want to identify the laneways that could most benefit from planting to reduce overland water runoff as part of the Greening Your Laneway program.

2. As a building manager or residence owner, I want to visualise the potential of laneway greening in reducing water damage to my property from overflow events.

<div class="usecase-section-header">Learning Objectives</div>

At the end of this use case you will be able to:
* Work with spatial databases using Geopandas
* Visualise spatial data on an interactive map

<div class="usecase-section-header">Greening Laneways to Reduce Runoff</div>

##### **What is the Green Your Laneway Program?**

The green laneways program was established in 2017 with aa pilot project of four Laneways: Katherine Place, Meyers Place, Guildford Lane and Coromandel Place. These laneways were greened using a mixture of potted plants on windowsills and nature strips as well as creeper vines. The benefits of greening laneways includes:

- oxygen production for fresh air to breathe
- storage of carbon dioxide, helping reduce green house gases
- trees absorb water, helping reduce flooding events
- evapotranspiration from leaves produces a cooling effect, reducing heat waves
- provide habitats for wildlife
- trees reduce cortisol in our brains, improving mental health and wellbeing

More information:
https://www.treesforcities.org/stories/trees-in-our-cities-10-reasons-we-need-to-plant-more


##### **How do plants reduce surface water overflow?**

Plants and soil will absorb some of the excess water during rain events. One problem that occurs within Melboune city is the runoff of water over roads. Due to non-permeable surfaces, water runs across the top of the land prior to enterinng a stormwater drain, causing "rivers" of water on roads and laneways in the CBD.  For this reason, consideration of the locations where water flows over land should be investigated in regards to choosing laneway locations for future greening projects.

<div class="usecase-section-header">Relevant datasets</div>

[Laneways with Greening Potential](https://data.melbourne.vic.gov.au/explore/dataset/laneways-with-greening-potential/export/)
- The City of Melbourne collected data analysing the best potential type of plantings in each of the city laneways; farm plantings, park species, vertically growing species and forest type species. Each type of planting was given a categorical value of potential for types of planting: Highest, Good, Some or Lowest.

[Water flow routes over land (Urban Forest)](https://data.melbourne.vic.gov.au/explore/dataset/water-flow-routes-over-land-urban-forest/export/)
- Data was collected in 2008 using an ESRI Spatial Analyst stream order tool. 

<div class="usecase-section-header">Exploratory data analysis steps</div>

1. Import modules
2. Access and read in data sets as Geopandas files
3. Map overlap locations of water runoff and laneways



In [4]:
# Import modules
import requests
import pandas as pd
import geopandas

In [5]:
# Function to get data from website using API
def get_data_single(base, SPECIFIC_PATH, apikey, offset=0):    
    # Set the filters, limit retrieves 20 rows at a time, offset says where to start data collection
    filters = f'records?limit={100}&offset={offset}&timezone=UTC'
    # Make the url from base, data url and filters variables stored ouside loop
    url = f'{base}{SPECIFIC_PATH}/{filters}&apikey={apikey}'
    # print(url) - can be used locally to check if code is working if 404 error given
    # Use the requests function to get the data
    result = requests.get(url)
    # Check that the request works, error code 200 = successful
    if result.status_code == 200:
        # Save results as a json file
        result_json = result.json()
        # Store a variable of max_results with total of dataset
        max_results = result_json['total_count']
        # Save the results key data to a list variable
        records = result_json['results']
    else:
        # If data is not collected correctly return the error
        print("ERROR GETTING DATA: ", result.status_code)
        max_results = 0
        records = []
    # At end of function, return the json results in records, max_results count and offset
    return [records, max_results, offset]


def fetch_all_data(SPECIFIC_PATH):
    # Collect data from API
    # Set offset increment
    # (needs to match offset in get data function)
    OFFSET_INCREMENT = 100
    # Base url (this should be the same for all datasets)
    base_url = 'https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/'
    # Set your API key
    # Set path to your API key
    # (needs to be a destination on your local device where API is kept)
    path = 'C:/Users/alito/Documents/SIT764/MOP-Code/env.txt'
    apikey_local = open(path, 'r')
    apikey = apikey_local.read()   
    # Call the get data function, passing in variables above, save to result
    result = get_data_single(base_url, SPECIFIC_PATH, apikey)
    # Save the records data returned in the get_data function to records list variable
    records = result[0]
    # Save the dataset size data returned in the get_data function to max_results variable
    max_results = result[1] 
    # Increase the offset returned in the get_data function (result[2]) by the offset increment
    offset = result[2] + OFFSET_INCREMENT
    # Check the length of the data returned and compare it against the max_results variable
    # If the length o fthe data is less than the max_results, run the while loop
    while len(records) != max_results:
        # Call the get data function again, passing in url, specific path and new offset value
        data = get_data_single(base_url, SPECIFIC_PATH, apikey, offset)
        # Add the data collected to the existing records list
        records += data[0]
        # Increase the offset by the offset increment
        offset += OFFSET_INCREMENT
    # Convert the records list of dictionaries into a pandas dataframe 
    df = pd.DataFrame(records)
    # Print the dataframe
    return df

In [11]:
# Read in Laneways with Greening Potential Dataset
# Set specific url 
SPECIFIC_PATH = 'laneways-with-greening-potential'
# Collect data
lane_df = fetch_all_data(SPECIFIC_PATH)     
lane_df.head(2)

,geo_point_2d,geo_shape,segid_1,mapbase_mc,insol_scor,verticalsu,segid,driveways,parklane,objectid,...,farmlane,walls,farm_rank,vert_rank,fores_rank,forest,park_rank,binsperct,mapbase_1,wsud
0,"{'lon': 144.9623447558369, 'lat': -37.81980050...","{'type': 'Feature', 'geometry': {'coordinates'...",10714,10714.0,6.492595,0.0,10714,1,53.830526,1,...,58.518386,Probably Unsuitable,Highest Potential,Lowest potential,Lowest potential,0.000000,Highest potential,0.0,Banana Alley,Probably Unsuitable
1,"{'lon': 144.97170834375964, 'lat': -37.8109763...","{'type': 'Feature', 'geometry': {'coordinates'...",20229,20229.0,2.215613,0.0,20229,2,0.000000,5,...,0.000000,Probably Unsuitable,Lowest potential,Lowest potential,Good potential,49.153357,Lowest potential,60.0,Harwood Place,Consider


In [12]:
# Read in Water flow routes over land (Urban Forest) Dataset
# Set specific url 
SPECIFIC_PATH = 'water-flow-routes-over-land-urban-forest'
# Collect data
water_df = fetch_all_data(SPECIFIC_PATH)     
water_df.head(2)


,geo_point_2d,geo_shape,grid_code,source,to_node,arcid,from_node
0,"{'lon': 144.93012309946076, 'lat': -37.8091117...","{'type': 'Feature', 'geometry': {'coordinates'...",3,2008 DEM to stream order using ESRI Spatial An...,3112,3502,3111
1,"{'lon': 144.9260809159682, 'lat': -37.80905866...","{'type': 'Feature', 'geometry': {'coordinates'...",1,2008 DEM to stream order using ESRI Spatial An...,3117,3510,3116
